In [2]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "bn")

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to /home/user/.cache/huggingface/datasets/wikiann/bn/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
label_names = dataset["train"].features["ner_tags"].feature.names

In [5]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
})

In [6]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
    tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
    #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used 
    #so the new keys [input_ids, labels (after adjustment)]
    #can be added to the datasets dict for each train test validation split
    print('tokenized_samples')
    print(tokenized_samples)
    total_adjusted_labels = []
    print(len(tokenized_samples["input_ids"]))
    print('-' * 50)
    for k in range(0, len(tokenized_samples["input_ids"])):
        prev_wid = -1
        word_ids_list = tokenized_samples.word_ids(batch_index=k)
        existing_label_ids = all_samples_per_split["ner_tags"][k]
        i = -1
        adjusted_label_ids = []
   
        for wid in word_ids_list:
            if(wid is None):
                adjusted_label_ids.append(-100)
            elif(wid!=prev_wid):
                i = i + 1
                adjusted_label_ids.append(existing_label_ids[i])
                prev_wid = wid
            else:
                label_name = label_names[existing_label_ids[i]]
                adjusted_label_ids.append(existing_label_ids[i])
        
        total_adjusted_labels.append(adjusted_label_ids)
    tokenized_samples["labels"] = total_adjusted_labels
    return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)
print(tokenized_dataset)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

1000


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

1000


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: sejinp98. Use `wandb login --relogin` to force relogin


acc,▁▅▇▇██▇█
loss,▆█▆▃▃▃▃▁
acc,0.83111
loss,0.16237


# NER Fine tuning

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("basakdemirok/my_furniture_ner_model")

model = AutoModelForTokenClassification.from_pretrained("basakdemirok/my_furniture_ner_model", from_tf=True)

2023-05-20 20:37:05.028993: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 20:37:05.889342: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-20 20:37:07.340737: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-20 20:37:07.340972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
from transformers import pipeline

recognizer = pipeline("ner", model=model, tokenizer=tokenizer)

In [7]:
recognizer('whitechair')

[{'entity': 'B-FUR',
  'score': 0.67430717,
  'index': 1,
  'word': 'white',
  'start': 0,
  'end': 5},
 {'entity': 'I-FUR',
  'score': 0.49156886,
  'index': 2,
  'word': '##chai',
  'start': 5,
  'end': 9}]

In [19]:
import re

re.split(r'\t', 'a\t black\t chair')

['a', ' black', ' chair']

# Test Fine Tuning NER Model

In [1]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [2]:
print(texts[0][10:17], tags[0][10:17], sep='\n')

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']


In [3]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [4]:
len(train_texts[0])

22

In [5]:
len(val_texts[0])

13

In [6]:
len(train_tags[0])

22

In [7]:
len(val_tags[0])

13

In [8]:
val_texts[0]

['@aplusk',
 'What',
 'made',
 'you',
 'laugh',
 '?',
 'This',
 'did',
 ',',
 'courtesy',
 'of',
 '@rainnwilson',
 'http://www.officetally.com/rainn-wilson-and-friends-seattle-oct-23']

In [9]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [10]:
tag2id

{'B-person': 0,
 'B-corporation': 1,
 'B-creative-work': 2,
 'I-group': 3,
 'B-product': 4,
 'B-location': 5,
 'I-location': 6,
 'I-creative-work': 7,
 'O': 8,
 'I-person': 9,
 'I-corporation': 10,
 'I-product': 11,
 'B-group': 12}

In [11]:
id2tag

{0: 'B-person',
 1: 'B-corporation',
 2: 'B-creative-work',
 3: 'I-group',
 4: 'B-product',
 5: 'B-location',
 6: 'I-location',
 7: 'I-creative-work',
 8: 'O',
 9: 'I-person',
 10: 'I-corporation',
 11: 'I-product',
 12: 'B-group'}

In [12]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

ImportError: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with OpenSSL 1.1.0h  27 Mar 2018. See: https://github.com/urllib3/urllib3/issues/2168

In [ ]:
for i in range(len(val_texts)):
    print(len(val_texts[i]))

In [ ]:
val_texts[0]

In [ ]:
train_texts[0]

In [ ]:
len(train_encodings[0])

In [ ]:
len(val_encodings[0])

In [ ]:
val_encodings[0]

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
train_labels

In [ ]:
labels = [[tag2id[tag] for tag in doc] for doc in tags]
labels

In [ ]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

In [ ]:
train_dataset

In [ ]:
from Preprocessing import *

train_dataset, val_dataset, unique_tags = get_dataset()

In [ ]:
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
torch.save(model.state_dict(), './results/model.h5')

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model.load_state_dict(torch.load('./results/model.h5'))
model.eval()

In [ ]:
from transformers import pipeline
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
recognizer = pipeline("ner", model=model, tokenizer=tokenizer)

recognizer('the building')

In [ ]:
from torch.utils.data import DataLoader
from transformers import DistilBertForTokenClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        try:
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            loss.backward()
            optim.step()
        except:
            print(len(input_ids))
            print(len(attention_mask))
            print(len(labels))

model.eval()

In [ ]:
train_encodings.__getitem__

In [ ]:
len(train_encodings.__getitem__('input_ids'))
# input_ids
# attention_mask
# labels

In [ ]:
len(train_encodings.__getitem__('attention_mask'))

In [ ]:
len(train_encodings.__getitem__('labels'))

In [ ]:
train_dataset

In [ ]:
from torch.utils.data import DataLoader
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

train_loader = DataLoader(train_dataset, batch_size=15, shuffle=True)

num = 0
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    print(len(input_ids))
    print(len(attention_mask))
    print(len(labels))
    print('-' * 70)
    num += 1

print(num)

In [ ]:
input_ids.shape

In [ ]:
attention_mask.shape

In [17]:
labels.shape

torch.Size([16, 90])

In [31]:
len(train_encodings[0])

90

In [32]:
len(train_labels[0])

90

In [41]:
len(val_encodings[0])

78

In [37]:
len(val_labels[1])

78

In [38]:
val_encodings[0]

Encoding(num_tokens=78, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [39]:
val_encodings[1]

Encoding(num_tokens=78, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [41]:
len(train_encodings)

3

In [44]:
len(val_encodings)

3

In [45]:
for i in range(len(train_encodings)):
    print(i)

0
1
2


In [46]:
for i in range(len(val_encodings)):
    print(i)

0
1
2


In [47]:
for i in range(len(train_encodings)):
    if len(train_encodings[i]) != 90:
        print(i)
        print(train_encodings[i])
    else :
        print(len(train_encodings[i]))

90
90
90


In [43]:
for i in range(len(val_encodings)):
    if len(val_encodings[i]) != 79:
        print(i)
        print(val_encodings[i])
    else :
        print(len(val_encodings[i]))

79
79
79


In [48]:
for i in range(len(train_labels)):
    if len(train_labels[i]) != 90:
        print(i)
        print(train_labels[i])
    else :
        print(len(train_labels[i]))

90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
90
9

# Test NER Processing

In [1]:
from TreatSentence import NER

In [2]:
ner_model = "DeveloperSejin/NER_for_furniture_3D_object_create"
ner = NER(ner_model)

In [3]:
ner.return_dic(' A white velvet upholstered armchair with curved armrests: This luxurious armchair is upholstered in a silky white velvet fabric and has curved armrests that provide additional comfort and support. The chair has a sturdy wooden frame and is perfect for a cozy reading corner or a luxurious bedroom.')[0]

{'entity': 'O',
 'score': 0.9998679,
 'index': 1,
 'word': 'A',
 'start': 1,
 'end': 2}

In [4]:
ner.get_tag('A white velvet upholstered armchair with curved armrests: This luxurious armchair is upholstered in a silky white velvet fabric and has curved armrests that provide additional comfort and support. The chair has a sturdy wooden frame and is perfect for a cozy reading corner or a luxurious bedroom.')

{'B-color',
 'B-design',
 'B-furniture',
 'B-material',
 'I-furniture',
 'I-material',
 'O'}

In [1]:
ner.get_missing_tags('A white velvet upholstered armchair with curved armrests: This luxurious armchair is upholstered in a silky white velvet fabric and has curved armrests that provide additional comfort and support. The chair has a sturdy wooden frame and is perfect for a cozy reading corner or a luxurious bedroom.')

NameError: name 'ner' is not defined

In [1]:
from Getprompt import gpt

ner_model = "DeveloperSejin/NER_for_furniture_3D_object_create"
gpt = gpt('', ner_model)

[nltk_data] Downloading package wordnet to /home/sejin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
prompt = 'A white velvet upholstered armchair with curved armrests: This luxurious armchair is upholstered in a silky white velvet fabric and has curved armrests that provide additional comfort and support. The chair has a sturdy wooden frame and is perfect for a cozy reading corner or a luxurious bedroom.'
gpt.getAnswer(prompt)

1. A plush loveseat with tufted detailing: This cozy loveseat comes with cushions upholstered in a soft, velvety material that will make you never want to leave. Its tufted detailing adds a touch of elegance and sophistication to any space.

2. An oversized ottoman with hidden storage: This ottoman not only serves as a comfortable footrest but also has a hidden storage compartment to keep your living room clutter-free. Its oversized design makes it perfect for extra seating or as a makeshift coffee table.

3. A mid-century modern-inspired sofa with sleek lines: This sofa has a minimalist design that channels mid-century modern style. Its sharp, clean lines and tapered legs lend an air of sophistication to any living room.

4. A classic wingback chair in bold paisley print: This classic wingback chair gets a modern update with a bold paisley print. Its high back and slightly angled arms provide a comfortable and stylish seating option for any room.

5. A rustic farmhouse bench with dist

'{"recommend": "-size\\n\\n-design\\n\\n", "detail": {"detail0": {"prompt": "1. A plush loveseat with tufted detailing", "detail": "This cozy loveseat comes with cushions upholstered in a soft, velvety material that will make you never want to leave. Its tufted detailing adds a touch of elegance and sophistication to any space."}, "detail1": {"prompt": "2. An oversized ottoman with hidden storage", "detail": "This ottoman not only serves as a comfortable footrest but also has a hidden storage compartment to keep your living room clutter-free. Its oversized design makes it perfect for extra seating or as a makeshift coffee table."}, "detail2": {"prompt": "3. A mid-century modern-inspired sofa with sleek lines", "detail": "This sofa has a minimalist design that channels mid-century modern style. Its sharp, clean lines and tapered legs lend an air of sophistication to any living room."}, "detail3": {"prompt": "4. A classic wingback chair in bold paisley print", "detail": "This classic win

In [4]:
len([])

0